# Model 1 Capstone Project : Klasifikasi Gambar Menggunakan Convolutional Neural Networks (CNN)
Notebook ini bertujuan untuk membangun dan melatih model **Convolutional Neural Network (CNN)** untuk melakukan **klasifikasi gambar organik dan anorganik**. Langkah-langkah yang dilakukan meliputi persiapan data, pembuatan model, pelatihan, dan evaluasi model. Model CNN yang dibangun akan digunakan untuk mengklasifikasikan gambar sampah rumah tangga ke dalam dua kategori utama: **organik** dan **anorganik**.

# Nama Proyek : WasteSnap
Proyek inovatif berbasis website yang bertujuan untuk membantu masyarakat Indonesia mengidentifikasi dan mengelola sampah rumah tangga secara mandiri melalui platform berbasis web.

# Tujuan Proyek:
- **Mengidentifikasi Sampah Organik dan Non-Organik**: Dengan menggunakan model CNN yang dilatih pada dataset gambar, pengguna dapat mengidentifikasi jenis sampah secara otomatis.
- **Meningkatkan Kesadaran Lingkungan**: Proyek ini bertujuan untuk meningkatkan kesadaran masyarakat mengenai pengelolaan sampah dan pentingnya pemisahan sampah organik dan non-organik untuk mendukung program daur ulang.
- **Pengelolaan Sampah yang Lebih Efisien**: Dengan aplikasi ini, masyarakat dapat memisahkan sampah mereka dengan lebih mudah dan efisien, serta membantu mengurangi dampak lingkungan yang disebabkan oleh pembuangan sampah sembarangan.

Melalui proyek ini, teknologi klasifikasi gambar diharapkan dapat membantu masyarakat Indonesia dalam pengelolaan sampah rumah tangga secara lebih mandiri dan ramah lingkungan.

# Anggota Tim Machine Learning
## ID tim : CC25-CF318. 
Arthur Setiawan Waruwu | MC319D5Y2042 | Universitas Sumatera Utara | \
Sakifa Indira Putri | MC319D5X2380 | Universitas Sumatera Utara | \
Diva Anggreini Harahap | MC319D5X2329 | Universitas Sumatera Utara |

# Penjelasan Dataset
Dataset ini berisi gambar-gambar sampah rumah tangga yang sudah kami kelompokkan dalam folder yang sesuai dengan jenisnya. Dataset ini memiliki 13.8k gambar, yang masing-masing kami kumpulkan secara mandiri dari berbagai sumber, seperti Kaggle, Google Images, Pinterest, dan sumber-sumber lainnya. Berikut adalah kategori-kategori yang terdapat dalam dataset ini:
- Buah
- Cangkang Telur
- Elektronik
- Kaca
- Kain
- Kardus
- Karet
- Kayu
- Kertas
- Kotoran Hewan
- Logam
- Plastik
- Sayuran
- Sepatu
- Sisa Teh dan Kopi
- Sisa Makanan
- Styrofoam

# 1. Import Libraries
Langkah pertama yang akan kita lakukan adalah mengimpor berbagai libraries yang menyediakan fungsi-fungsi yang kita perlukan.

In [ ]:
# Install tensorflow menggunakan perintah pip
!pip install tensorflow opencv-python Pillow tensorflowjs

In [ ]:
# Import Library yang diperlukan
# Untuk pengolahan citra
import cv2
from PIL import Image, ImageEnhance  

# Untuk operasi array dan manipulasi data
import numpy as np
import pandas as pd
import os
import random
import shutil

# Untuk visualisasi
import matplotlib.pyplot as plt  # Matplotlib

# Untuk machine learning / deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv2D, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,  ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split  
import tensorflowjs as tfjs
from tensorflow.keras.models import Model

# 2. Data Preparation
Persiapan data untuk klasifikasi gambar melibatkan pengolahan gambar sehingga dapat dimasukkan ke dalam model CNN.

## Data Loading
Pada bagian ini, kita akan memuat dataset gambar yang akan digunakan untuk train dan validation model. Dataset telah diupload ke kaggle dan akan dimuat langsung ke dalam notebook. Sebelum itu, kita akan menyalin dataset dari direktori `/kaggle/input/` yang sifatnya itu read-only ke direktori working `/kaggle/working/` yang dapat ditulis, jadi kita bisa melakukan berbagai operasi tanpa terbatas oleh izin akses read-only.

In [ ]:
dataset_dir = '/kaggle/input/datasetcapstonefixx/Dataset-Asli'

working_dir = '/kaggle/working/dataset-capstone'

if not os.path.exists(dataset_dir):
    print(f"Direktori sumber {dataset_dir} tidak ditemukan!")
else:
    if os.path.exists(working_dir):
        print(f"Folder {working_dir} sudah ada, sedang dihapus...")
        shutil.rmtree(working_dir) 

    # menyalin dataset dari /kaggle/input ke /kaggle/working yang bisa ditulis
    try:
        shutil.copytree(dataset_dir, working_dir)
        print(f"Dataset berhasil disalin ke {working_dir}")
    except Exception as e:
        print(f"Gagal menyalin dataset: {e}")

In [ ]:
# menghitung total gambar di dataset
def count_images_per_class_in_main_folders(base_dir, main_folders):
    total_images = 0  
    
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            print(f"Folder {main_folder} tidak ditemukan di {base_dir}")
            continue
        
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        print(f"\nFolder utama '{main_folder}' memiliki {len(kelas_list)} kelas:")
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            images = [f for f in os.listdir(kelas_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            jumlah = len(images)
            print(f"  Kelas '{kelas}' : {jumlah} gambar")
            total_images += jumlah
    
    print(f"\nTotal seluruh gambar di semua kelas: {total_images}")

main_folders = ['Anorganik', 'Organik']
count_images_per_class_in_main_folders(working_dir, main_folders)

In [ ]:
# memeriksa ekstensi gambar yang valid
valid_extensions = {'.jpg', '.jpeg', '.png'}

def cek_ekstensi_tidak_valid(base_dir, main_folders):
    invalid_files = []
    
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            print(f"Folder {main_folder} tidak ditemukan di {base_dir}")
            continue
        
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            files = os.listdir(kelas_path)
            
            for f in files:
                ext = os.path.splitext(f)[1].lower()
                if ext not in valid_extensions:
                    invalid_files.append(os.path.join(main_folder, kelas, f))
    
    if invalid_files:
        print("File dengan ekstensi tidak valid ditemukan:")
        for file_path in invalid_files:
            print(f" - {file_path}")
        print(f"\nTotal file dengan ekstensi tidak valid: {len(invalid_files)}")
    else:
        print("Semua file memiliki ekstensi gambar yang valid.")
        print("Total file tidak valid: 0")

# Contoh pemanggilan fungsi
main_folders = ['Anorganik', 'Organik']
cek_ekstensi_tidak_valid(working_dir, main_folders)

Melihat dari output di atas, tampaknya masih ada sedikit gambar dengan ekstensi .webp yang tidak valid. Oleh karena itu, kita akan menghapus gambar-gambar ini terlebih dahulu sebelum membagi dataset.

In [ ]:
# menghapus gambar yang tidak valid
def hapus_file_tidak_valid(base_dir, main_folders):
    deleted_files = []
    
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            print(f"Folder {main_folder} tidak ditemukan di {base_dir}")
            continue
        
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            files = os.listdir(kelas_path)
            
            for f in files:
                ext = os.path.splitext(f)[1].lower()
                if ext not in valid_extensions:
                    file_path = os.path.join(kelas_path, f)
                    try:
                        os.remove(file_path)
                        deleted_files.append(os.path.join(main_folder, kelas, f))
                    except Exception as e:
                        print(f"Gagal menghapus {file_path}: {e}")
    
    if deleted_files:
        print("Berhasil menghapus file berikut dengan ekstensi tidak valid:")
        for file_path in deleted_files:
            print(f" - {file_path}")
        print(f"\nTotal file yang dihapus: {len(deleted_files)}")
    else:
        print("Tidak ditemukan file dengan ekstensi tidak valid untuk dihapus.")

main_folders = ['Anorganik', 'Organik']
hapus_file_tidak_valid(working_dir, main_folders)

Pada tahap ini, telah dilakukan pengecekan dan penghapusan terhadap gambar-gambar yang tidak valid dalam dataset. Berdasarkan hasil pemeriksaan, sebanyak **34 gambar** dengan ekstensi tidak valid telah dihapus.

In [ ]:
def cek_ekstensi_tidak_valid(base_dir, main_folders):
    invalid_files = []
    
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            continue
        
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            files = os.listdir(kelas_path)
            
            for f in files:
                ext = os.path.splitext(f)[1].lower()
                if ext not in valid_extensions:
                    invalid_files.append(os.path.join(main_folder, kelas, f))
    
    if invalid_files:
        print("Masih terdapat file dengan ekstensi tidak valid:")
        for file_path in invalid_files:
            print(f" - {file_path}")
    else:
        print("Tidak ditemukan file dengan ekstensi tidak valid lagi.")

main_folders = ['Anorganik', 'Organik']
cek_ekstensi_tidak_valid(working_dir, main_folders)

Nah, bisa dipastikan bahwa seluruh gambar dalam dataset ini sudah **valid** dan siap digunakan untuk pelatihan model.

In [ ]:
def show_sample_images(base_dir, main_folders):
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            print(f"Folder {main_folder} tidak ditemukan.")
            continue
        
        print(f"\n--- Contoh gambar dari kelas di folder '{main_folder}' ---")
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        
        images = []
        titles = []
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            files = [f for f in os.listdir(kelas_path) if os.path.splitext(f)[1].lower() in valid_extensions]
            
            if len(files) == 0:
                print(f"Tidak ada gambar valid di kelas '{kelas}'")
                continue
            
            contoh_gambar_path = os.path.join(kelas_path, files[0])
            img = cv2.imread(contoh_gambar_path)
            if img is None:
                print(f"Gagal membaca gambar {contoh_gambar_path}")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            titles.append(f"{main_folder} - {kelas}")
        
        if len(images) == 0:
            print(f"Tidak ada gambar valid untuk ditampilkan di folder '{main_folder}'.")
            continue
        
        cols = 4
        total = len(images)
        rows = (total + cols - 1) // cols  

        fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5)) 
        axes = axes.flatten() if total > 1 else [axes]

        for i in range(len(axes)):
            if i < total:
                axes[i].imshow(images[i])
                axes[i].set_title(titles[i], fontsize=14)
                axes[i].axis('off')
            else:
                axes[i].axis('off') 
        
        plt.tight_layout()
        plt.show()

# Contoh pemanggilan fungsi
main_folders = ['Anorganik', 'Organik']
show_sample_images(working_dir, main_folders)

## Split Dataset

In [ ]:
# split data 80% Train, 10% Test, 10% Val

working_dir = '/kaggle/working/dataset-capstone'  
output_dir = '/kaggle/working/Dataset-Split'

main_folders = ['Anorganik', 'Organik']
splits = {
    'Train': 0.8,
    'Test': 0.1,
    'Val': 0.1
}

valid_extensions = ('.jpg', '.jpeg', '.png')

random.seed(42)  

def create_dir_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)

def split_dataset(base_dir, output_dir, main_folders, splits):
   
    for split_name in splits.keys():
        for main_folder in main_folders:
            create_dir_if_not_exist(os.path.join(output_dir, split_name, main_folder))
    
    for main_folder in main_folders:
        main_path = os.path.join(base_dir, main_folder)
        if not os.path.isdir(main_path):
            print(f"Folder {main_folder} tidak ditemukan, skip.")
            continue
        
        kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
        
        for kelas in kelas_list:
            kelas_path = os.path.join(main_path, kelas)
            files = [f for f in os.listdir(kelas_path) if f.lower().endswith(valid_extensions)]
            
            random.shuffle(files)
            
            n_total = len(files)
            n_train = int(n_total * splits['Train'])
            n_test = int(n_total * splits['Test'])
            n_val = n_total - n_train - n_test
            
            train_files = files[:n_train]
            test_files = files[n_train:n_train+n_test]
            val_files = files[n_train+n_test:]
            
            for split_name, file_list in zip(['Train', 'Test', 'Val'], [train_files, test_files, val_files]):
                kelas_output_path = os.path.join(output_dir, split_name, main_folder, kelas)
                create_dir_if_not_exist(kelas_output_path)
                
                for file_name in file_list:
                    src_path = os.path.join(kelas_path, file_name)
                    dst_path = os.path.join(kelas_output_path, file_name)
                    shutil.copy2(src_path, dst_path)
            
            print(f"{main_folder}/{kelas}: total={n_total}, train={len(train_files)}, test={len(test_files)}, val={len(val_files)}")

split_dataset(working_dir, output_dir, main_folders, splits)

Dapat dilihat output diatas merupakan hasil split data dengan rasio Train 80, Test 10, dan Val 10

In [ ]:
# menghitung jumlah gambar train, val, dan test
def count_images_in_split(output_dir, main_folders, splits):
    print("\nTotal jumlah gambar per split:")
    for split_name in splits.keys():
        total_count = 0
        split_path = os.path.join(output_dir, split_name)
        for main_folder in main_folders:
            main_path = os.path.join(split_path, main_folder)
            if not os.path.isdir(main_path):
                continue
            kelas_list = [k for k in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, k))]
            for kelas in kelas_list:
                kelas_path = os.path.join(main_path, kelas)
                files = [f for f in os.listdir(kelas_path) if f.lower().endswith(valid_extensions)]
                total_count += len(files)
        print(f"{split_name}: {total_count} gambar")

count_images_in_split(output_dir, main_folders, splits)


## Data Preprocessing

In [ ]:
# Augmentasi, Resize dan Normalisasi untuk folder train
from skimage import exposure

# custom contrast 
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

# path folder train setelah split
train_folder = os.path.join(output_dir, 'Train')

train_datagen = ImageDataGenerator(
    rescale=1./255,               # normalisasi
    rotation_range=30,            # rotasi
    width_shift_range=0.2,        # translasi horizontal
    height_shift_range=0.2,       # translasi vertikal
    shear_range=0.3,              # shearing
    zoom_range=0.3,               # zoom
    horizontal_flip=True,         # flip horizontal
    fill_mode='nearest',
    channel_shift_range=50,       # perpindahan warna channel
    ##preprocessing_function=contrast_stretching  # fungsi contrast stretching
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),  # resize gambar ke ukuran 224x224
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    seed=42
)

print("Folder 'train' berhasil diproses dengan ImageDataGenerator.")
print(f"Jumlah kelas: {len(train_generator.class_indices)}")

In [ ]:
# Normalisasi untuk folder Test dan Validation
test_folder = os.path.join(output_dir, 'Test')
val_folder = os.path.join(output_dir, 'Val')

val_test_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_test_datagen.flow_from_directory(
    val_folder,
    batch_size=32,
    target_size=(224, 224),  # ini juga harus sama
    class_mode='categorical',
    shuffle=False,
    color_mode= 'rgb'
)

test_generator = val_test_datagen.flow_from_directory(
    test_folder,
    batch_size=32,
    target_size=(224, 224),  # ini juga harus sama
    class_mode='categorical',
    shuffle=False,
    color_mode= 'rgb'
)

# Output informasi sukses untuk validation
print("Folder 'validation' berhasil diproses dengan ImageDataGenerator.")
print(f"Jumlah kelas (val): {len(val_generator.class_indices)}")

# Output informasi sukses untuk test
print("Folder 'test' berhasil diproses dengan ImageDataGenerator.")
print(f"Jumlah kelas (test): {len(test_generator.class_indices)}")

In [ ]:
print(f"Mapping kelas: {train_generator.class_indices}")
print(f"Mapping kelas (val): {val_generator.class_indices}")
print(f"Mapping kelas (test): {test_generator.class_indices}")

# **3. Modelling**

In [ ]:
import numpy as np
from collections import Counter

# Menghitung jumlah data per label
label_counts = Counter(train_generator.classes)

# Mengambil mapping label
label_map = train_generator.class_indices
label_map_inv = {v: k for k, v in label_map.items()}

# Menampilkan hasil
for label_index, count in label_counts.items():
    print(f"{label_map_inv[label_index]}: {count} gambar")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ambil semua label dari generator
train_labels = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

In [ ]:
from tensorflow.keras.applications import MobileNetV2

# Input layer
input_tensor = Input(shape=(224, 224, 3))

# Base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)
print("Jumlah total layer:", len(base_model.layers))

# Set semua layer dapat dilatih terlebih dahulu
base_model.trainable = True

# Bekukan semua layer kecuali 50 terakhir
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Tambahkan custom layers di atas base model
x = base_model.output
x = Conv2D(8, (3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
output_tensor = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Buat model final
model = Model(inputs=input_tensor, outputs=output_tensor)

# Lihat arsitektur model
model.summary()

In [ ]:

# Callback ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

# Callback EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,             # jika val_loss tidak membaik selama 5 epoch, training stop
    restore_best_weights=True,
    verbose=1
)

# Compile dan fit dengan epochs 50 dan callbacks
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[reduce_lr, early_stop],
    class_weight=class_weights_dict if class_weights_dict else None
)


# **4. Evaluasi Model dan Visualisasi**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# ===== 1. VISUALISASI =====
# Menggabungkan history1 dan history2 jika kamu mau, tapi di sini hanya history2
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))

# Plot Akurasi
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training vs Validation Accuracy')

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training vs Validation Loss')

plt.show()

In [ ]:
# Ambil prediksi model
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification Report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# **5. Konversi Model**

In [ ]:
model.save("model_organik_anorganik.h5")

In [ ]:
from IPython.display import FileLink
FileLink('model_organik_anorganik.h5')  # klik link yang muncul untuk download

In [ ]:
!pip install tf2onnx

In [ ]:
import tf2onnx
import tensorflow as tf

# Konversi model ke ONNX
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)

# Simpan model ONNX
with open("model_organik_anorganik.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())



In [ ]:
from IPython.display import FileLink
FileLink('model_organik_anorganik.onnx')


In [ ]:
!pip install tensorflowjs


# **6. Inference**